In [7]:
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel, write_stan_json
import arviz as az
import matplotlib.pyplot as plt
import preliz as pz
az.style.use("arviz-grayscale")

In [8]:
def get_ig_params(x_vals, l_b=None, u_b=None, mass=0.95, plot=False):
    """
    Returns a weakly informative prior for the length-scale parameter of the GP kernel.
    """

    differences = np.abs(np.subtract.outer(x_vals, x_vals))
    if l_b is None:
        l_b = np.min(differences[differences != 0]) * 2
    if u_b is None:
        u_b = np.max(differences) / 1.5

    dist = pz.InverseGamma()
    pz.maxent(dist, l_b, u_b, mass, plot=plot)

    return dict(zip(dist.param_names, dist.params))

In [9]:
bikes = pd.read_csv("https://raw.githubusercontent.com/aloctavodia/BAP3/main/code/data/bikes.csv")
y = bikes["rented"].values
X = bikes["hour"].values

In [10]:
ivg_prior = get_ig_params(X)

/home/harrison/anaconda3/envs/cmdstanpyenv/lib/python3.12/site-packages/scipy/stats/_distn_infrastructure.py:2069: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


In [11]:
model = CmdStanModel(stan_file="HSGP.stan", cpp_options={'STAN_THREADS': 'TRUE'},
                     force_compile = True)

18:05:05 - cmdstanpy - INFO - compiling stan file /home/harrison/Desktop/gitHubRepos/HPCStatsPortfolio/PythonProjects/Bayesian_stats/gaussianprocesses/HSGP.stan to exe file /home/harrison/Desktop/gitHubRepos/HPCStatsPortfolio/PythonProjects/Bayesian_stats/gaussianprocesses/HSGP
18:05:32 - cmdstanpy - INFO - compiled model executable: /home/harrison/Desktop/gitHubRepos/HPCStatsPortfolio/PythonProjects/Bayesian_stats/gaussianprocesses/HSGP


In [12]:
data = {'N': len(y),
        'x': X,
        'y': y,
        "c_f": 1.5,
        "M_f": 10,
        'alpha_prior': ivg_prior["alpha"],
        'beta_prior': ivg_prior["beta"]}
write_stan_json("/home/harrison/Desktop/gitHubRepos/HPCStatsPortfolio/PythonProjects/Bayesian_stats/gaussianprocesses/data.json", data = data)

In [13]:
fit = model.sample("data.json", chains = 4 , iter_sampling=1000, parallel_chains = 4)

18:05:32 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status





chain 1 |▉         | 00:00 Iteration:    1 / 2000 [  0%]  (Warmup)


chain 1 |█▎        | 00:01 Iteration:  100 / 2000 [  5%]  (Warmup)


chain 1 |█▊        | 00:01 Iteration:  200 / 2000 [ 10%]  (Warmup)


chain 1 |██▎       | 00:01 Iteration:  300 / 2000 [ 15%]  (Warmup)


chain 1 |██▋       | 00:02 Iteration:  400 / 2000 [ 20%]  (Warmup)


chain 1 |███▏      | 00:02 Iteration:  500 / 2000 [ 25%]  (Warmup)


chain 1 |███▋      | 00:02 Iteration:  600 / 2000 [ 30%]  (Warmup)


chain 1 |████      | 00:02 Iteration:  700 / 2000 [ 35%]  (Warmup)


chain 1 |█████     | 00:03 Iteration:  900 / 2000 [ 45%]  (Warmup)


chain 1 |█████▉    | 00:03 Iteration: 1001 / 2000 [ 50%]  (Sampling)


chain 1 |██████▎   | 00:03 Iteration: 1100 / 2000 [ 55%]  (Sampling)


chain 1 |██████▊   | 00:03 Iteration: 1200 / 2000 [ 60%]  (Sampling)


chain 1 |███████▎  | 00:04 Iteration: 1300 / 2000 [ 65%]  (Sampling)


c


18:05:38 - cmdstanpy - INFO - CmdStan done processing.
18:05:38 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: neg_binomial_2_lpmf: Precision parameter is 0, but must be positive finite! (in 'HSGP.stan', line 47, column 0 to column 62)
	Exception: neg_binomial_2_lpmf: Precision parameter is 0, but must be positive finite! (in 'HSGP.stan', line 47, column 0 to column 62)
	Exception: neg_binomial_2_lpmf: Precision parameter is 0, but must be positive finite! (in 'HSGP.stan', line 47, column 0 to column 62)
	Exception: neg_binomial_2_lpmf: Location parameter[1] is -nan, but must be positive finite! (in 'HSGP.stan', line 47, column 0 to column 62)
	Exception: neg_binomial_2_lpmf: Precision parameter is 0, but must be positive finite! (in 'HSGP.stan', line 47, column 0 to column 62)
	Exception: neg_binomial_2_lpmf: Precision parameter is 0, but must be positive finite! (in 'HSGP.stan', line 47, column 0 to column 62)
	Exception: neg_binomial_2_lpmf: Precision parameter

In [18]:
cmdstanpy_data = az.from_cmdstanpy(
    posterior=fit   
)

In [ ]:
posterior_islands = cmdstanpy_data.posterior.stack(samples=("chain", "draw"))
trace_η = posterior_islands['eta'].values
trace_ℓ = posterior_islands['length'].values

_, ax = plt.subplots(1, 1, figsize=(11, 4))
xrange = np.linspace(0, 7)

median_ = np.median(trace_η[:,None] * (np.exp(-xrange**2 / (2*trace_ℓ[:,None]**2))), axis=0)

ax.plot(xrange, median_, lw=3)


ax.plot(xrange, (trace_η[::20][:, None] * np.exp(-xrange**2 / (2*trace_ℓ[::20][:, None]**2))).T,
        'C0', alpha=.1)

ax.set_ylim(0, 1)
ax.set_xlabel('distance (thousand kilometers)')
ax.set_ylabel('covariance')
plt.savefig('../fig/GP_islands_dist_cov.png')